# 最急降下法の実装

このノートブックでは、線形回帰における最急降下法を段階的に実装していきます。

## 目次
1. [Step 1: データの準備](#step-1-データの準備)
2. [Step 2: 目的関数の実装](#step-2-目的関数の実装)
3. [Step 3: 勾配の計算](#step-3-勾配の計算)
4. [Step 4: パラメータ更新](#step-4-パラメータ更新)
5. [Step 5: 収束判定](#step-5-収束判定)
6. [Step 6: 完全なアルゴリズム](#step-6-完全なアルゴリズム)
7. [実データでの検証](#実データでの検証)
8. [演習問題](#演習問題)


In [ ]:
# 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 乱数の固定
np.random.seed(42)

print("ライブラリのインポートが完了しました。")


## Step 1: データの準備

まず、線形回帰のためのサンプルデータを生成します。


In [ ]:
# サンプルデータの生成
# 住宅価格予測の例：面積から価格を予測
np.random.seed(42)

# 住宅の面積（m²）
area = np.random.uniform(30, 120, 100)

# 価格（万円）= 50万円/m² + ノイズ
price = 50 * area + np.random.normal(0, 100, 100)

# データフレームに変換
data = pd.DataFrame({
    'area': area,
    'price': price
})

print("データの基本統計:")
print(data.describe())
print(f"\nデータ形状: {data.shape}")

# データの可視化
plt.figure(figsize=(10, 6))
plt.scatter(data['area'], data['price'], alpha=0.6, color='blue')
plt.xlabel('面積 (m²)')
plt.ylabel('価格 (万円)')
plt.title('住宅価格データ')
plt.grid(True, alpha=0.3)
plt.show()

# 相関係数の確認
correlation = data['area'].corr(data['price'])
print(f"\n面積と価格の相関係数: {correlation:.3f}")


In [ ]:
# データの前処理
# 特徴量（X）と目的変数（y）に分割
X = data[['area']].values  # 2次元配列として保持
y = data['price'].values

# データの標準化（最急降下法では重要）
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

print("標準化前のデータ:")
print(f"X - 平均: {X.mean():.2f}, 標準偏差: {X.std():.2f}")
print(f"y - 平均: {y.mean():.2f}, 標準偏差: {y.std():.2f}")

print("\n標準化後のデータ:")
print(f"X_scaled - 平均: {X_scaled.mean():.2f}, 標準偏差: {X_scaled.std():.2f}")
print(f"y_scaled - 平均: {y_scaled.mean():.2f}, 標準偏差: {y_scaled.std():.2f}")

# バイアス項（切片）の追加
X_with_bias = np.c_[np.ones(X_scaled.shape[0]), X_scaled]
print(f"\nバイアス項追加後の特徴量行列の形状: {X_with_bias.shape}")


## Step 2: 目的関数の実装

最急降下法で最小化する目的関数（平均二乗誤差）を実装します。


In [ ]:
def compute_cost(X, y, theta):
    """
    平均二乗誤差（MSE）を計算する関数
    
    Parameters:
    X : array-like, shape (n_samples, n_features)
        特徴量行列
    y : array-like, shape (n_samples,)
        目的変数
    theta : array-like, shape (n_features,)
        回帰係数（パラメータ）
    
    Returns:
    cost : float
        平均二乗誤差
    """
    m = len(y)  # サンプル数
    predictions = X.dot(theta)  # 予測値の計算
    cost = (1 / (2 * m)) * np.sum((predictions - y) ** 2)
    return cost

# 初期パラメータでの目的関数の値を確認
initial_theta = np.zeros(X_with_bias.shape[1])  # [0, 0]で初期化
initial_cost = compute_cost(X_with_bias, y_scaled, initial_theta)

print(f"初期パラメータ: {initial_theta}")
print(f"初期コスト（目的関数の値）: {initial_cost:.4f}")

# 異なるパラメータでの目的関数の値を確認
test_theta1 = np.array([0, 1])  # 切片=0, 傾き=1
test_theta2 = np.array([1, 0])  # 切片=1, 傾き=0

cost1 = compute_cost(X_with_bias, y_scaled, test_theta1)
cost2 = compute_cost(X_with_bias, y_scaled, test_theta2)

print(f"\nパラメータ [0, 1] でのコスト: {cost1:.4f}")
print(f"パラメータ [1, 0] でのコスト: {cost2:.4f}")


## Step 3: 勾配の計算

目的関数の勾配（偏微分）を計算する関数を実装します。


In [ ]:
def compute_gradient(X, y, theta):
    """
    目的関数の勾配を計算する関数
    
    Parameters:
    X : array-like, shape (n_samples, n_features)
        特徴量行列
    y : array-like, shape (n_samples,)
        目的変数
    theta : array-like, shape (n_features,)
        回帰係数（パラメータ）
    
    Returns:
    gradient : array-like, shape (n_features,)
        勾配ベクトル
    """
    m = len(y)  # サンプル数
    predictions = X.dot(theta)  # 予測値の計算
    error = predictions - y  # 誤差
    gradient = (1 / m) * X.T.dot(error)  # 勾配の計算
    return gradient

# 初期パラメータでの勾配を確認
initial_gradient = compute_gradient(X_with_bias, y_scaled, initial_theta)
print(f"初期パラメータでの勾配: {initial_gradient}")

# 勾配の意味を理解するための可視化
def plot_cost_surface():
    """目的関数の表面を可視化"""
    # パラメータの範囲を設定
    theta0_range = np.linspace(-2, 2, 50)
    theta1_range = np.linspace(-2, 2, 50)
    
    # グリッドを作成
    Theta0, Theta1 = np.meshgrid(theta0_range, theta1_range)
    
    # 各点でのコストを計算
    costs = np.zeros_like(Theta0)
    for i in range(len(theta0_range)):
        for j in range(len(theta1_range)):
            theta = np.array([Theta0[i, j], Theta1[i, j]])
            costs[i, j] = compute_cost(X_with_bias, y_scaled, theta)
    
    # 等高線プロット
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    contour = plt.contour(Theta0, Theta1, costs, levels=20)
    plt.clabel(contour, inline=True, fontsize=8)
    plt.xlabel('θ₀ (切片)')
    plt.ylabel('θ₁ (傾き)')
    plt.title('目的関数の等高線')
    plt.colorbar()
    
    plt.subplot(1, 2, 2)
    plt.contourf(Theta0, Theta1, costs, levels=20, cmap='viridis')
    plt.xlabel('θ₀ (切片)')
    plt.ylabel('θ₁ (傾き)')
    plt.title('目的関数の表面（カラーマップ）')
    plt.colorbar()
    
    plt.tight_layout()
    plt.show()

plot_cost_surface()


## Step 4: パラメータ更新

勾配を使ってパラメータを更新する関数を実装します。


In [ ]:
def update_parameters(theta, gradient, learning_rate):
    """
    パラメータを更新する関数
    
    Parameters:
    theta : array-like, shape (n_features,)
        現在のパラメータ
    gradient : array-like, shape (n_features,)
        勾配ベクトル
    learning_rate : float
        学習率
    
    Returns:
    new_theta : array-like, shape (n_features,)
        更新されたパラメータ
    """
    new_theta = theta - learning_rate * gradient
    return new_theta

# 学習率の影響を確認
learning_rates = [0.01, 0.1, 0.5, 1.0]
theta_test = np.array([0.0, 0.0])
gradient_test = np.array([0.5, -0.3])

print("異なる学習率でのパラメータ更新:")
for lr in learning_rates:
    new_theta = update_parameters(theta_test, gradient_test, lr)
    print(f"学習率 {lr:4.2f}: θ = [{new_theta[0]:6.3f}, {new_theta[1]:6.3f}]")

# 学習率の影響を可視化
def visualize_learning_rate_effect():
    """学習率の影響を可視化"""
    theta_start = np.array([1.5, 1.5])
    gradient = np.array([-0.5, -0.3])
    
    plt.figure(figsize=(15, 3))
    
    for i, lr in enumerate([0.1, 0.5, 1.0, 2.0]):
        plt.subplot(1, 4, i+1)
        
        # パラメータの更新軌跡
        theta = theta_start.copy()
        trajectory = [theta.copy()]
        
        for _ in range(10):
            theta = update_parameters(theta, gradient, lr)
            trajectory.append(theta.copy())
        
        trajectory = np.array(trajectory)
        
        plt.plot(trajectory[:, 0], trajectory[:, 1], 'bo-', markersize=4)
        plt.plot(trajectory[0, 0], trajectory[0, 1], 'go', markersize=8, label='開始点')
        plt.plot(trajectory[-1, 0], trajectory[-1, 1], 'ro', markersize=8, label='終了点')
        
        plt.xlabel('θ₀')
        plt.ylabel('θ₁')
        plt.title(f'学習率 = {lr}')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

visualize_learning_rate_effect()


## Step 5: 収束判定

最急降下法の収束を判定する条件を実装します。


In [ ]:
def check_convergence(old_theta, new_theta, old_cost, new_cost, 
                     theta_tolerance=1e-6, cost_tolerance=1e-6):
    """
    収束判定を行う関数
    
    Parameters:
    old_theta : array-like
        前回のパラメータ
    new_theta : array-like
        現在のパラメータ
    old_cost : float
        前回のコスト
    new_cost : float
        現在のコスト
    theta_tolerance : float
        パラメータ変化の許容値
    cost_tolerance : float
        コスト変化の許容値
    
    Returns:
    converged : bool
        収束したかどうか
    """
    # パラメータの変化量
    theta_change = np.linalg.norm(new_theta - old_theta)
    
    # コストの変化量
    cost_change = abs(new_cost - old_cost)
    
    # 収束判定
    converged = (theta_change < theta_tolerance) or (cost_change < cost_tolerance)
    
    return converged, theta_change, cost_change

# 収束判定のテスト
theta_old = np.array([1.0, 0.5])
theta_new = np.array([1.0001, 0.5001])
cost_old = 0.1234
cost_new = 0.1233

converged, theta_change, cost_change = check_convergence(
    theta_old, theta_new, cost_old, cost_new
)

print(f"パラメータ変化量: {theta_change:.6f}")
print(f"コスト変化量: {cost_change:.6f}")
print(f"収束判定: {converged}")

# 収束判定の可視化
def plot_convergence_criteria():
    """収束判定の基準を可視化"""
    iterations = np.arange(1, 21)
    
    # シミュレーション：コストが指数的に減少
    costs = 1.0 * np.exp(-0.3 * iterations) + 0.01
    
    # パラメータ変化量のシミュレーション
    theta_changes = 0.1 * np.exp(-0.2 * iterations) + 0.001
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.semilogy(iterations, costs, 'bo-', label='コスト')
    plt.axhline(y=1e-6, color='r', linestyle='--', label='収束閾値 (1e-6)')
    plt.xlabel('イテレーション')
    plt.ylabel('コスト')
    plt.title('コストの収束')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.semilogy(iterations, theta_changes, 'go-', label='パラメータ変化量')
    plt.axhline(y=1e-6, color='r', linestyle='--', label='収束閾値 (1e-6)')
    plt.xlabel('イテレーション')
    plt.ylabel('パラメータ変化量')
    plt.title('パラメータの収束')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_convergence_criteria()


## Step 6: 完全なアルゴリズム

これまでに実装した関数を組み合わせて、完全な最急降下法アルゴリズムを実装します。


In [ ]:
def gradient_descent(X, y, learning_rate=0.01, max_iterations=1000, 
                    theta_tolerance=1e-6, cost_tolerance=1e-6, verbose=False):
    """
    最急降下法の完全な実装
    
    Parameters:
    X : array-like, shape (n_samples, n_features)
        特徴量行列
    y : array-like, shape (n_samples,)
        目的変数
    learning_rate : float
        学習率
    max_iterations : int
        最大イテレーション数
    theta_tolerance : float
        パラメータ変化の許容値
    cost_tolerance : float
        コスト変化の許容値
    verbose : bool
        詳細な出力をするかどうか
    
    Returns:
    theta : array-like
        最適化されたパラメータ
    cost_history : list
        コストの履歴
    theta_history : list
        パラメータの履歴
    """
    # パラメータの初期化
    theta = np.zeros(X.shape[1])
    
    # 履歴の記録
    cost_history = []
    theta_history = [theta.copy()]
    
    # 初期コストの計算
    current_cost = compute_cost(X, y, theta)
    cost_history.append(current_cost)
    
    if verbose:
        print(f"初期パラメータ: {theta}")
        print(f"初期コスト: {current_cost:.6f}")
        print("-" * 50)
    
    # 最急降下法のメインループ
    for iteration in range(max_iterations):
        # 勾配の計算
        gradient = compute_gradient(X, y, theta)
        
        # パラメータの更新
        old_theta = theta.copy()
        theta = update_parameters(theta, gradient, learning_rate)
        
        # 新しいコストの計算
        new_cost = compute_cost(X, y, theta)
        
        # 履歴の記録
        cost_history.append(new_cost)
        theta_history.append(theta.copy())
        
        # 収束判定
        converged, theta_change, cost_change = check_convergence(
            old_theta, theta, current_cost, new_cost, 
            theta_tolerance, cost_tolerance
        )
        
        if verbose and (iteration + 1) % 100 == 0:
            print(f"イテレーション {iteration + 1:4d}: "
                  f"コスト = {new_cost:.6f}, "
                  f"パラメータ変化 = {theta_change:.6f}, "
                  f"コスト変化 = {cost_change:.6f}")
        
        # 収束した場合
        if converged:
            if verbose:
                print(f"\n収束しました！ (イテレーション {iteration + 1})")
                print(f"最終パラメータ: {theta}")
                print(f"最終コスト: {new_cost:.6f}")
            break
        
        current_cost = new_cost
    
    # 最大イテレーションに達した場合
    if iteration == max_iterations - 1:
        if verbose:
            print(f"\n最大イテレーション数 ({max_iterations}) に達しました。")
    
    return theta, cost_history, theta_history

# 最急降下法の実行
print("最急降下法の実行:")
print("=" * 50)

theta_optimal, cost_history, theta_history = gradient_descent(
    X_with_bias, y_scaled, 
    learning_rate=0.1, 
    max_iterations=1000,
    verbose=True
)


In [ ]:
# 結果の可視化
def plot_gradient_descent_results():
    """最急降下法の結果を可視化"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. コストの収束
    axes[0, 0].plot(cost_history)
    axes[0, 0].set_xlabel('イテレーション')
    axes[0, 0].set_ylabel('コスト')
    axes[0, 0].set_title('コストの収束')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. パラメータの収束
    theta_history = np.array(theta_history)
    axes[0, 1].plot(theta_history[:, 0], label='θ₀ (切片)')
    axes[0, 1].plot(theta_history[:, 1], label='θ₁ (傾き)')
    axes[0, 1].set_xlabel('イテレーション')
    axes[0, 1].set_ylabel('パラメータ値')
    axes[0, 1].set_title('パラメータの収束')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. パラメータ空間での軌跡
    axes[1, 0].plot(theta_history[:, 0], theta_history[:, 1], 'bo-', markersize=4)
    axes[1, 0].plot(theta_history[0, 0], theta_history[0, 1], 'go', markersize=8, label='開始点')
    axes[1, 0].plot(theta_history[-1, 0], theta_history[-1, 1], 'ro', markersize=8, label='終了点')
    axes[1, 0].set_xlabel('θ₀ (切片)')
    axes[1, 0].set_ylabel('θ₁ (傾き)')
    axes[1, 0].set_title('パラメータ空間での軌跡')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 最終的な回帰直線
    # 元のスケールに戻す
    theta_original_scale = theta_optimal.copy()
    theta_original_scale[0] = scaler_y.inverse_transform([[theta_optimal[0]]])[0, 0]
    theta_original_scale[1] = theta_optimal[1] * scaler_y.scale_[0] / scaler_X.scale_[0]
    
    # 予測直線の描画
    x_range = np.linspace(data['area'].min(), data['area'].max(), 100)
    y_pred = theta_original_scale[0] + theta_original_scale[1] * x_range
    
    axes[1, 1].scatter(data['area'], data['price'], alpha=0.6, color='blue', label='データ')
    axes[1, 1].plot(x_range, y_pred, 'r-', linewidth=2, label='回帰直線')
    axes[1, 1].set_xlabel('面積 (m²)')
    axes[1, 1].set_ylabel('価格 (万円)')
    axes[1, 1].set_title('最終的な回帰直線')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_gradient_descent_results()

# 結果の詳細表示
print(f"\n最適化されたパラメータ:")
print(f"切片 (β₀): {theta_optimal[0]:.6f}")
print(f"傾き (β₁): {theta_optimal[1]:.6f}")

print(f"\n収束までのイテレーション数: {len(cost_history) - 1}")
print(f"最終コスト: {cost_history[-1]:.6f}")
print(f"初期コスト: {cost_history[0]:.6f}")
print(f"コスト改善率: {(cost_history[0] - cost_history[-1]) / cost_history[0] * 100:.2f}%")


## 実データでの検証

scikit-learnの線形回帰と比較して、実装の正確性を検証します。


In [ ]:
# scikit-learnとの比較
from sklearn.linear_model import LinearRegression

# scikit-learnの線形回帰
lr_sklearn = LinearRegression()
lr_sklearn.fit(X_scaled, y_scaled)

# 結果の比較
print("実装の比較:")
print("=" * 50)
print(f"最急降下法 - 切片: {theta_optimal[0]:.6f}, 傾き: {theta_optimal[1]:.6f}")
print(f"scikit-learn - 切片: {lr_sklearn.intercept_:.6f}, 傾き: {lr_sklearn.coef_[0]:.6f}")

# 予測の比較
y_pred_gd = X_with_bias.dot(theta_optimal)
y_pred_sklearn = lr_sklearn.predict(X_scaled)

# 予測精度の比較
mse_gd = mean_squared_error(y_scaled, y_pred_gd)
mse_sklearn = mean_squared_error(y_scaled, y_pred_sklearn)

print(f"\n予測精度の比較:")
print(f"最急降下法 - MSE: {mse_gd:.6f}")
print(f"scikit-learn - MSE: {mse_sklearn:.6f}")
print(f"差: {abs(mse_gd - mse_sklearn):.8f}")

# 学習率の影響を調査
def compare_learning_rates():
    """異なる学習率での性能比較"""
    learning_rates = [0.001, 0.01, 0.1, 0.5, 1.0]
    
    plt.figure(figsize=(15, 5))
    
    for i, lr in enumerate(learning_rates):
        theta_lr, cost_hist, _ = gradient_descent(
            X_with_bias, y_scaled, 
            learning_rate=lr, 
            max_iterations=1000,
            verbose=False
        )
        
        plt.subplot(1, 3, 1)
        plt.plot(cost_hist, label=f'LR={lr}')
        plt.xlabel('イテレーション')
        plt.ylabel('コスト')
        plt.title('学習率の影響')
        plt.legend()
        plt.grid(True, alpha=0.3)
    
    # 収束速度の比較
    convergence_iterations = []
    for lr in learning_rates:
        _, cost_hist, _ = gradient_descent(
            X_with_bias, y_scaled, 
            learning_rate=lr, 
            max_iterations=1000,
            verbose=False
        )
        convergence_iterations.append(len(cost_hist) - 1)
    
    plt.subplot(1, 3, 2)
    plt.bar(range(len(learning_rates)), convergence_iterations)
    plt.xlabel('学習率のインデックス')
    plt.ylabel('収束までのイテレーション数')
    plt.title('収束速度の比較')
    plt.xticks(range(len(learning_rates)), [f'{lr}' for lr in learning_rates])
    plt.grid(True, alpha=0.3)
    
    # 最終コストの比較
    final_costs = []
    for lr in learning_rates:
        _, cost_hist, _ = gradient_descent(
            X_with_bias, y_scaled, 
            learning_rate=lr, 
            max_iterations=1000,
            verbose=False
        )
        final_costs.append(cost_hist[-1])
    
    plt.subplot(1, 3, 3)
    plt.bar(range(len(learning_rates)), final_costs)
    plt.xlabel('学習率のインデックス')
    plt.ylabel('最終コスト')
    plt.title('最終コストの比較')
    plt.xticks(range(len(learning_rates)), [f'{lr}' for lr in learning_rates])
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return learning_rates, convergence_iterations, final_costs

learning_rates, iterations, costs = compare_learning_rates()

# 結果のまとめ
print(f"\n学習率の影響:")
print("学習率 | 収束イテレーション | 最終コスト")
print("-" * 40)
for lr, iter_count, final_cost in zip(learning_rates, iterations, costs):
    print(f"{lr:6.3f} | {iter_count:15d} | {final_cost:10.6f}")


## 演習問題

### 問題1: 学習率の最適化
異なる学習率で最急降下法を実行し、最適な学習率を見つけてください。

### 問題2: 収束判定の改善
現在の収束判定を改善し、より効率的なアルゴリズムを作成してください。

### 問題3: バッチサイズの影響
ミニバッチ最急降下法を実装し、バッチサイズの影響を調査してください。

### 問題4: 正則化の追加
L1正則化（Lasso）とL2正則化（Ridge）を最急降下法に追加してください。



## まとめ

このノートブックでは、最急降下法の完全な実装を行いました：

### 実装した機能
1. **目的関数**: 平均二乗誤差の計算
2. **勾配計算**: 偏微分による勾配の導出
3. **パラメータ更新**: 学習率を使った反復的更新
4. **収束判定**: パラメータとコストの変化量による判定
5. **完全なアルゴリズム**: 全ての要素を組み合わせた実装

### 重要なポイント
- **データの標準化**: 最急降下法では必須
- **学習率の選択**: 収束速度と安定性のバランス
- **収束判定**: 適切な閾値の設定
- **可視化**: アルゴリズムの動作理解

### 次のステップ
- [正規方程式の実装](./normal_equation_implementation.ipynb)
- [基本的な線形回帰](./linear_regression_basic.ipynb)
- [特徴量スケーリング](../02_feature_scaling/02_feature_scaling.md)
